# Generate Topics and launch the web app

In [1]:
import os
import subprocess
import webbrowser
import time
from pathlib import Path
import json
from pathlib import Path
from bunkatopics.functions.web import launch_web_app

2023-10-20 12:17:04.717192: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 12:17:04.717230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 12:17:04.717264: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 12:17:04.725824: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 12:17:05.508652: W tensorflow/compiler/

# Load documents and topics

In [2]:
documents_path = Path('json_examples/docs.jsonl')
topics_path = Path('json_examples/topics.jsonl')
topics = []
documents = []
# Open the JSONL files and load data
with open(topics_path, 'r') as serialized_topics:
    for line in serialized_topics:
        # Parse each line as a JSON object and append it to the list
        data = json.loads(line)
        topics.append(data)

with open(documents_path, 'r') as serialized_documents:
    for line in serialized_documents:
        # Parse each line as a JSON object and append it to the list
        data = json.loads(line)
        documents.append(data)

source_data = {
    'documents': documents,
    'topics': topics
}

In [3]:
launch_web_app(source_data)


/home/elishowk/src/BunkaTopics/web/public/localSearchResults.json
export NEXT_PUBLIC_API_ENDPOINT=local
cd web && npm i && npm run start

up to date, audited 432 packages in 3s

118 packages are looking for funding
  run `npm fund` for details

4 vulnerabilities (2 moderate, 2 high)

To address all issues possible, run:
  npm audit fix --force

Some issues need review, and may require choosing
a different dependency.

Run `npm audit` for details.

> bunkatopicsweb@0.1.0 prestart
> next build



warn  - You are using a non-standard "NODE_ENV" value in your environment. This creates inconsistencies in the project and is strongly advised against. Read more: https://nextjs.org/docs/messages/non-standard-node-env


info  - Loaded env from /home/elishowk/src/BunkaTopics/web/.env.local
info  - Linting and checking validity of types...


info  - Creating an optimized production build...
info  - Compiled successfully
info  - Collecting page data...
info  - Generating static pages (0/10)
next-translate - compiled page: / - locale: en - namespaces: common - used loader: getStaticProps
next-translate - compiled page: / - locale: fr - namespaces: common - used loader: getStaticProps
info  - Generating static pages (2/10)
info  - Generating static pages (4/10)
info  - Generating static pages (7/10)
next-translate - compiled page: /search - locale: fr - namespaces: common - used loader: getStaticProps
next-translate - compiled page: /search - locale: en - namespaces: common - used loader: getStaticProps
info  - Generating static pages (10/10)
info  - Finalizing page optimization...

Route (pages)                              Size     First Load JS
┌ ● /                                      9.32 kB         165 kB
├   /_app                                  0 B             104 kB
├ ○ /404                                   182 B 

warn  - You are using a non-standard "NODE_ENV" value in your environment. This creates inconsistencies in the project and is strongly advised against. Read more: https://nextjs.org/docs/messages/non-standard-node-env
warn  - You are using a non-standard "NODE_ENV" value in your environment. This creates inconsistencies in the project and is strongly advised against. Read more: https://nextjs.org/docs/messages/non-standard-node-env
